In [1]:
from shallue_vanderburg_util.preprocess import *
import matplotlib.pyplot as plt
import pandas as pd
import lightkurve as lk
from lightkurve import *

In [2]:
TCE_DIR = "dr24_tce_full.csv"
KEPLER_DATA_DIR = "../kepler/"
FIG_DIM = (20, 5)
# a 6022556
# b 7978202
KEPID = 6022556

In [3]:
TCE_DIR = "dr24_tce_full.csv"
tce_df = pd.read_csv(TCE_DIR, skiprows=159)
tce = tce_df[tce_df.kepid == KEPID].iloc[0]
period = tce["tce_period"]
duration = tce["tce_duration"]
t0 = tce["tce_time0bk"]

In [1]:
from shallue_vanderburg_util.astronet.train import train
train()

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [ ]:
all_time, all_flux = read_light_curve(KEPID, KEPLER_DATA_DIR)

In [ ]:
time, flux = process_light_curve(all_time, all_flux)

In [ ]:
generate_example_for_tce(time, flux, tce)

In [ ]:
f = 3/0

In [ ]:
plt.figure(figsize=FIG_DIM)
plt.scatter(time, flux)
plt.show()

In [ ]:
folded_time, folded_values = phase_fold_and_sort_light_curve(time, flux, period, t0)

In [ ]:
def reject_outliers(data, m = 30.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    mask = s>m
    return np.where(mask, np.nan, data)

plt.figure(figsize=FIG_DIM)
minus_outliers = reject_outliers(folded_values)
plt.scatter(folded_time, minus_outliers)
plt.show()


In [ ]:
def scatter_lg(d, t):
    plt.figure(figsize=FIG_DIM)
    plt.scatter(np.arange(-len(d)/2, len(d)/2), d, label="TCE {}".format(KEPID))
    plt.title(t)
    plt.xlabel("Phase-folded datapoints, centered at t0")
    plt.ylabel("Normalized flux")
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
g = global_view(folded_time, folded_values, period)
scatter_lg(g, "Global View")

In [ ]:
l = local_view(folded_time, folded_values, period, duration)
scatter_lg(l, "Local View")